<a href="https://colab.research.google.com/github/hkhong72/big_data/blob/main/221101_6_Chap19(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras import models, layers
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [ ]:
# 생성자 모델 만들기
generator = models.Sequential()
generator.add(layers.Dense(128*7*7, input_dim=100, activation=layers.LeakyReLU(0.2)))
generator.add(layers.BatchNormalization()) # 스케일자체를 축소(0~1사이로)
generator.add(layers.Reshape((7, 7, 128))) # 7by7이미지가 128개채널 -> 3차원으로변경
generator.add(layers.UpSampling2D()) 
# MaxPooling에 반대. width, heights 두배씩, 총면적 -> 4배
generator.add(layers.Conv2D(64, kernel_size=5, padding='same')) 
#입력이미지와 출력이미지에 사이즈변화X, 2by2씩 붙여야함
generator.add(layers.BatchNormalization()) 
generator.add(layers.Activation(layers.LeakyReLU(0.2))) 
generator.add(layers.UpSampling2D())
generator.add(layers.Conv2D(1, kernel_size=5, padding='same', activation='tanh'))
generator.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 6272)              633472    
                                                                 
 batch_normalization_3 (Batc  (None, 6272)             25088     
 hNormalization)                                                 
                                                                 
 reshape_1 (Reshape)         (None, 7, 7, 128)         0         
                                                                 
 up_sampling2d_2 (UpSampling  (None, 14, 14, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        204864    
                                                                 
 batch_normalization_4 (Batc  (None, 14, 14, 64)      

In [ ]:
# 1. 입력값100 * 128*7*7 + 128*7*7 = 633472
# 2. 입력값128*7*7(6272) * 4(?) = 25088
# 3. 128 * w1600 + 64 = 204864
# 4. 64 * w192 + 64 = 256
# 5. (64 + (28-3)) * 1 + 1 = 1601

In [ ]:
# 판별자 모델 만들기
discriminator = models.Sequential()
discriminator.add(layers.Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding="same"))
discriminator.add(layers.Activation(layers.LeakyReLU(0.2)))
discriminator.add(layers.Dropout(0.3))
discriminator.add(layers.Conv2D(128, kernel_size=5, strides=2, padding="same"))
discriminator.add(layers.Activation(layers.LeakyReLU(0.2)))
discriminator.add(layers.Dropout(0.3))
discriminator.add(layers.Flatten())
discriminator.add(layers.Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable=False # 별도학습을 시키기위해

In [ ]:
ginput = layers.Input(shape=(100,)) # 생성자는 처음에100개노이즈들어옴
dis_output = discriminator(generator(ginput))
gan = models.Model(ginput, dis_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')
gan.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential_2 (Sequential)   (None, 28, 28, 1)         865281    
                                                                 
 sequential_6 (Sequential)   (None, 1)                 212865    
                                                                 
Total params: 1,078,146
Trainable params: 852,609
Non-trainable params: 225,537
_________________________________________________________________


In [ ]:
# 신경망을 실행시키는 함수 만들기
def gan_train(epoch, batch_size, saving_interval):
    #MNIST epdlxj qnffjdhrl
    #이미지만 사용하기 때문에 X_train만 호출
    (X_train, _), (_, _) = mnist.load_data()
    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype(float)
    # 127.5를 뺀 후 127.5로 나누어서 -1 ~ 1 사이의 값으로 바꾼다.
    X_train = (X_train - 127.5) / 127.5 # 생성자처음 activation이 tanh이니까
    
    true = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    